## Random Forest Model for Kepler Space Telescope Data

The purpose of this project is to reinforce my data science and machine learning knowledge, and to help others in their journeys. I'm not an expert on these topics, and I'm conscious that there are a lot of tools, concepts and techniques that I need to master. That's why I develop this project, to share with the community the tools, concepts and techniques, that I have learned across my data science journey. I'm open to comments, critics and feedback that would help me to develop learn best practices and corrections in case I was wrong

In [1]:
#import warnings
#warnings.simplefilter('ignore')
import pandas as pd
import numpy as np

In [2]:
kepler_df = pd.read_csv('../../data/exoplanet_data.csv')
kepler_df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [3]:
kepler_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6991 entries, 0 to 6990
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   koi_disposition    6991 non-null   object 
 1   koi_fpflag_nt      6991 non-null   int64  
 2   koi_fpflag_ss      6991 non-null   int64  
 3   koi_fpflag_co      6991 non-null   int64  
 4   koi_fpflag_ec      6991 non-null   int64  
 5   koi_period         6991 non-null   float64
 6   koi_period_err1    6991 non-null   float64
 7   koi_period_err2    6991 non-null   float64
 8   koi_time0bk        6991 non-null   float64
 9   koi_time0bk_err1   6991 non-null   float64
 10  koi_time0bk_err2   6991 non-null   float64
 11  koi_impact         6991 non-null   float64
 12  koi_impact_err1    6991 non-null   float64
 13  koi_impact_err2    6991 non-null   float64
 14  koi_duration       6991 non-null   float64
 15  koi_duration_err1  6991 non-null   float64
 16  koi_duration_err2  6991 

In [4]:
kepler_df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

In [5]:
print(f"The dataframe has a length of {len(kepler_df.columns)} columns.")
print(f"There are 3 outcomes/predictions/targets: {set(kepler_df['koi_disposition'])}")

The dataframe has a length of 41 columns.
There are 3 outcomes/predictions/targets: {'CONFIRMED', 'FALSE POSITIVE', 'CANDIDATE'}


In [6]:
features = kepler_df.drop('koi_disposition', axis = 1)
target = kepler_df['koi_disposition'].values

print(features.shape)
print(target.shape)

(6991, 40)
(6991,)


### Non-Scaled

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.3, random_state = 42)

In [8]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (4893, 40)
Training Labels Shape: (4893,)
Testing Features Shape: (2098, 40)
Testing Labels Shape: (2098,)


In [9]:
#from sklearn.preprocessing import LabelEncoder

#label_encoder = LabelEncoder()
#label_encoder.fit(target)
#encoded_y_train = label_encoder.transform(y_train)
#encoded_y_test = label_encoder.transform(y_test)

In [10]:
#print(encoded_y_train)
#print(encoded_y_test)

In [11]:
#categories = label_encoder.inverse_transform([0, 1, 2])
#categories

In [13]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators= 1000, random_state=42)

rf.fit(X_train_scaled, y_train);

ValueError: could not convert string to float: 'CANDIDATE'

In [ ]:
prediction = rf.predict(X_test)
prediction

In [ ]:
non_scaled_df = pd.DataFrame({'Prediction':prediction, 'Actual':encoded_y_test}).reset_index(drop=True)
non_scaled_df.head(10)

In [ ]:
print(f"Non Scaled Score: {round(rf.score(X_test, encoded_y_test)*100,2)}%")

In [ ]:
important_features = list(rf.feature_importances_)
features_list = list(features.columns)
                          

In [ ]:
feature_importances = [(feature, round(importance, 3)) for feature, importance in zip(features_list, important_features)]
feature_importances

In [ ]:
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
[print('Feature: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
feat_list = []
values_list = []
for pair in feature_importances:
    feat_list.append(pair[0])
    values_list.append(pair[1])

In [ ]:
sorted_importances = np.sort(important_features)[::-1]
top_sort_importances = list(sorted_importances[0:10])

In [ ]:
import matplotlib.pyplot as plt

x_val = list(range(len(values_list[:10])))
fig, ax = plt.subplots(figsize = (10, 6), dpi=128)

ax.bar(feat_list[:10], values_list[:10], orientation = 'vertical', color='lightblue', edgecolor='blue')
ax.set_xticklabels(feat_list[:10], rotation=90)
ax.set_ylabel('Importance')
ax.set_xlabel('Variable')
ax.set_title('Variable Importances')

plt.savefig('non_scaled_random_forest_top_features.png')
plt.show()

In [ ]:
import plotly.express as px
from plotly.offline import offline

fig = px.bar(x=feat_list[:10], y=values_list[:10],
             color=values_list[:10],
             color_continuous_scale='blues')

fig.write_image("non_scaled_features_importance_bar_chart_randomforest.png")
offline.plot(fig, filename='non_scaled_features_importance_bar_chart_randomforest.html')
fig.show()

### Scaled

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
scaled_rf = RandomForestRegressor(n_estimators= 1000, random_state=42)
scaled_rf.fit(X_train_scaled, encoded_y_train);

In [ ]:
scaled_important_features = list(scaled_rf.feature_importances_)

In [ ]:
scaled_important_features

In [ ]:
scaled_feature_importances = [(feature, round(importance, 3)) for feature, importance in zip(features_list, scaled_important_features)]
scaled_feature_importances

In [ ]:
scaled_feature_importances = sorted(scaled_feature_importances, key = lambda x: x[1], reverse = True)
[print('Feature: {:20} Importance: {}'.format(*pair)) for pair in scaled_feature_importances];

In [ ]:
scaled_vs_non_scaled = pd.DataFrame({'Scaled Feature Importances':scaled_feature_importances, 'Feature Importances':feature_importances}).reset_index(drop=True)
scaled_vs_non_scaled.head(10)

In [ ]:
scfeat_list = []
scvalues_list = []
for pair in scaled_feature_importances:
    scfeat_list.append(pair[0])
    scvalues_list.append(pair[1])

In [ ]:
x_val = list(range(len(scvalues_list[:10])))
fig, ax = plt.subplots(figsize = (10, 6), dpi=128)

ax.bar(scfeat_list[:10], values_list[:10], orientation = 'vertical', color='salmon', edgecolor='red')
ax.set_xticklabels(scfeat_list[:10], rotation=90)
ax.set_ylabel('Importance')
ax.set_xlabel('Variable')
ax.set_title('Variable Importances')

plt.savefig('scaled_random_forest_top_features.png')
plt.show()

In [ ]:
fig = px.bar(x=scfeat_list[:10], y=scvalues_list[:10],
             color=scvalues_list[:10],
             color_continuous_scale='redor')

fig.write_image("scaled_features_importance_bar_chart_randomforest.png")
offline.plot(fig, filename='scaled_features_importance_bar_chart_randomforest.html')
fig.show()